In [ ]:
import pandas as pd
import numpy as np

wrk = './''
df = pd.read_csv('data/uci-epileptic/data.csv')
fake_df_normalized = pd.DataFrame(np.load('synthetic.npy'))

In [ ]:
fake_df = ((fake_df_normalized.drop(178, 1) * 4000) - 2000).round()

In [ ]:
fake_df["OUTPUT_LABEL"] = fake_df_normalized[178] >= 0.5
fake_df["OUTPUT_LABEL"] = fake_df["OUTPUT_LABEL"].astype(int)
fake_df.head()

In [ ]:
df["OUTPUT_LABEL"] = df.y == 1
df["OUTPUT_LABEL"] = df["OUTPUT_LABEL"].astype(int)
df.pop('y')
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

In [ ]:
def calc_prevalence(y_actual):
    # this function calculates the prevalence of the positive class (label = 1)
    return sum(y_actual) / len(y_actual)

print(
    'prevalence of the positive class: {:.4f} and {:.4f}'.format(
        calc_prevalence(df["OUTPUT_LABEL"].values),
        calc_prevalence(fake_df["OUTPUT_LABEL"].values)
    )
)

In [ ]:
print('# of Columns: {}, {}'.format(len(df.columns), len(fake_df.columns)))

In [ ]:
collist = df.columns.tolist()
cols_input = collist[0:178]
df_data = df[cols_input + ["OUTPUT_LABEL"]]

In [ ]:
# check for duplicated columns in cols_input
dup_cols = set([x for x in cols_input if cols_input.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0, "you have duplicated columns in cols_input"

# check for duplicated columns in df_data
cols_df_data = list(df_data.columns)
dup_cols = set([x for x in cols_df_data if cols_df_data.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0,'you have duplicated columns in df_data'

# check the size of df_data makes sense
assert (len(cols_input) + 1) == len(
    df_data.columns
), "issue with dimensions of df_data or cols_input"

In [ ]:
fake_collist = fake_df.columns.tolist()
fake_cols_input = fake_collist[0:178]
fake_df_data = fake_df[fake_cols_input + ["OUTPUT_LABEL"]]

In [ ]:
# check for duplicated columns in cols_input
fake_dup_cols = set([x for x in fake_cols_input if fake_cols_input.count(x) > 1])
print(fake_dup_cols)
assert len(fake_dup_cols) == 0, "you have duplicated columns in cols_input"

# check for duplicated columns in df_data
fake_cols_df_data = list(fake_df_data.columns)
fake_dup_cols = set([x for x in fake_cols_df_data if fake_cols_df_data.count(x) > 1])
print(fake_dup_cols)
assert len(fake_dup_cols) == 0,'you have duplicated columns in df_data'

# check the size of df_data makes sense
assert (len(fake_cols_input) + 1) == len(
    fake_df_data.columns
), "issue with dimensions of df_data or cols_input"

In [ ]:
df_data = df_data.sample(n=len(df_data))
df_data = df_data.reset_index(drop=True)

In [ ]:
fake_df_data = fake_df_data.sample(n=len(fake_df_data))
fake_df_data = fake_df_data.reset_index(drop=True)

In [ ]:
df_valid_test = df_data.sample(frac=0.3)
print("Validation/Test Split Size: %.1f" % (len(df_valid_test) / len(df_data)))

df_test = df_valid_test.sample(frac=0.5)
df_valid = df_valid_test.drop(df_test.index)

df_train_all = df_data.drop(df_valid_test.index)

In [ ]:
fake_df_valid_test = fake_df_data.sample(frac=0.3)
print("Validation/Test Split Size: %.1f" % (len(fake_df_valid_test) / len(fake_df_data)))

fake_df_test = fake_df_valid_test.sample(frac=0.5)
fake_df_valid = fake_df_valid_test.drop(fake_df_test.index)

fake_df_train_all = fake_df_data.drop(fake_df_valid_test.index)

In [ ]:
# check the prevalence of each
print(
    "Test prevalence(n = %d):%.3f"
    % (len(df_test), calc_prevalence(df_test.OUTPUT_LABEL.values))
)
print(
    "Valid prevalence(n = %d):%.3f"
    % (len(df_valid), calc_prevalence(df_valid.OUTPUT_LABEL.values))
)
print(
    "Train all prevalence(n = %d):%.3f"
    % (len(df_train_all), calc_prevalence(df_train_all.OUTPUT_LABEL.values))
)

In [ ]:
# check the prevalence of each
print(
    "Test prevalence(n = %d):%.3f"
    % (len(fake_df_test), calc_prevalence(fake_df_test.OUTPUT_LABEL.values))
)
print(
    "Valid prevalence(n = %d):%.3f"
    % (len(fake_df_valid), calc_prevalence(fake_df_valid.OUTPUT_LABEL.values))
)
print(
    "Train all prevalence(n = %d):%.3f"
    % (len(fake_df_train_all), calc_prevalence(fake_df_train_all.OUTPUT_LABEL.values))
)

In [ ]:
print('all samples (n = %d)'%len(df_data))
assert len(df_data) == (len(df_test)+len(df_valid)+len(df_train_all)),'math didnt work'

In [ ]:
print('all samples (n = %d)'%len(fake_df_data))
assert len(fake_df_data) == (len(fake_df_test)+len(fake_df_valid)+len(fake_df_train_all)), 'math didnt work'

In [ ]:
import numpy as np

rows_pos = df_train_all.OUTPUT_LABEL == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]

n = np.min([len(df_train_pos), len(df_train_neg)])

df_train = pd.concat([df_train_pos.sample(n=n, random_state=69), df_train_neg.sample(n=n, random_state=69)], axis=0, ignore_index=True)

df_train = df_train.sample(n=len(df_train), random_state=69).reset_index(drop=True)

print('Train balanced prevalence(n = %d):%.3f'%(len(df_train), calc_prevalence(df_train.OUTPUT_LABEL.values)))

In [ ]:
import numpy as np

rows_pos = fake_df_train_all.OUTPUT_LABEL == 1
fake_df_train_pos = fake_df_train_all.loc[rows_pos]
fake_df_train_neg = fake_df_train_all.loc[~rows_pos]

n = np.min([len(fake_df_train_pos), len(fake_df_train_neg)])

fake_df_train = pd.concat([fake_df_train_pos.sample(n=n, random_state=69), fake_df_train_neg.sample(n=n, random_state=69)], axis=0, ignore_index=True)

fake_df_train = fake_df_train.sample(n=len(fake_df_train), random_state=69).reset_index(drop=True)

print('Train balanced prevalence(n = %d):%.3f'%(len(fake_df_train), calc_prevalence(fake_df_train.OUTPUT_LABEL.values)))

In [ ]:
df_train_all.to_csv(wrk + 'data/processed/df_train_all.csv',index=False)
df_train.to_csv(wrk + 'data/processed/df_train.csv',index=False)
df_valid.to_csv(wrk + 'data/processed/df_valid.csv',index=False)
df_test.to_csv(wrk + 'data/processed/df_test.csv',index=False)

In [ ]:
fake_df_train_all.to_csv(wrk + 'data/processed/fake_df_train_all.csv',index=False)
fake_df_train.to_csv(wrk + 'data/processed/fake_df_train.csv',index=False)
fake_df_valid.to_csv(wrk + 'data/processed/fake_df_valid.csv',index=False)
fake_df_test.to_csv(wrk + 'data/processed/fake_df_test.csv',index=False)

In [ ]:
import pickle
pickle.dump(cols_input, open(wrk + 'data/processed/cols_input.sav', 'wb'))

In [ ]:
# a function to fill missing values with mean of the column if needed
def fill_my_missing(df, df_mean, col2use):
    # This function fills the missing values

    # check the columns are present
    for c in col2use:
        assert c in df.columns, c + ' not in df'
        assert c in df_mean.col.values, c+ 'not in df_mean'
    
    # replace the mean 
    for c in col2use:
        mean_value = df_mean.loc[df_mean.col == c,'mean_val'].values[0]
        df[c] = df[c].fillna(mean_value)
    return df

In [ ]:
# create the X and y matrices
X_train = df_train[cols_input].values
X_train_all = df_train_all[cols_input].values
X_valid = df_valid[cols_input].values

y_train = df_train['OUTPUT_LABEL'].values
y_valid = df_valid['OUTPUT_LABEL'].values

print('Training All shapes:',X_train_all.shape)
print('Training shapes:',X_train.shape, y_train.shape)
print('Validation shapes:',X_valid.shape, y_valid.shape)

from sklearn.preprocessing import StandardScaler
scaler  = StandardScaler()
scaler.fit(X_train_all)

scalerfile = wrk + 'data/processed/scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))
scaler = pickle.load(open(scalerfile, 'rb'))

# transform our data matrices
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)

In [ ]:
# create the X and y matrices
fake_X_train = fake_df_train[fake_cols_input].values
fake_X_train_all = fake_df_train_all[fake_cols_input].values
fake_X_valid = fake_df_valid[fake_cols_input].values

fake_y_train = fake_df_train['OUTPUT_LABEL'].values
fake_y_valid = fake_df_valid['OUTPUT_LABEL'].values

print('Training All shapes:',fake_X_train_all.shape)
print('Training shapes:',fake_X_train.shape, fake_y_train.shape)
print('Validation shapes:',fake_X_valid.shape, fake_y_valid.shape)

from sklearn.preprocessing import StandardScaler
fake_scaler  = StandardScaler()
fake_scaler.fit(fake_X_train_all)

fake_scalerfile = wrk + 'data/processed/fake_scaler.sav'
pickle.dump(fake_scaler, open(fake_scalerfile, 'wb'))
fake_scaler = pickle.load(open(fake_scalerfile, 'rb'))

# transform our data matrices
fake_X_train_tf = fake_scaler.transform(fake_X_train)
fake_X_valid_tf = fake_scaler.transform(fake_X_valid)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh):
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

In [ ]:
thresh = 0.5

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors = 100)
knn.fit(X_train_tf, y_train)

y_train_preds = knn.predict_proba(X_train_tf)[:,1]
y_valid_preds = knn.predict_proba(X_valid_tf)[:,1]

print('KNN')
print('Training:')
knn_train_auc, knn_train_accuracy, knn_train_recall, \
    knn_train_precision, knn_train_specificity = print_report(y_train,y_train_preds, thresh)
print('Validation:')
knn_valid_auc, knn_valid_accuracy, knn_valid_recall, \
    knn_valid_precision, knn_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
fake_knn=KNeighborsClassifier(n_neighbors = 100)
fake_knn.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_knn.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_knn.predict_proba(X_valid_tf)[:,1]

print('KNN')
print('Training:')
fake_knn_train_auc, fake_knn_train_accuracy, fake_knn_train_recall, \
    fake_knn_train_precision, fake_knn_train_specificity = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
fake_knn_valid_auc, fake_knn_valid_accuracy, fake_knn_valid_recall, \
    fake_knn_valid_precision, fake_knn_valid_specificity = print_report(y_valid, fake_y_valid_preds, thresh)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 6, random_state = 69)
rf.fit(X_train_tf, y_train)

y_train_preds = rf.predict_proba(X_train_tf)[:,1]
y_valid_preds = rf.predict_proba(X_valid_tf)[:,1]

print('Random Forest')
print('Training:')
rf_train_auc, rf_train_accuracy, rf_train_recall, rf_train_precision, \
rf_train_specificity =print_report(y_train,y_train_preds, thresh)
print('Validation:')
rf_valid_auc, rf_valid_accuracy, rf_valid_recall, rf_valid_precision, \
rf_valid_specificity = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
fake_rf = RandomForestClassifier(max_depth = 6, random_state = 69)
fake_rf.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_rf.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_rf.predict_proba(X_valid_tf)[:,1]

print('Random Forest')
print('Training:')
fake_rf_train_auc, fake_rf_train_accuracy, fake_rf_train_recall, fake_rf_train_precision, \
fake_rf_train_specificity = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
fake_rf_valid_auc, fake_rf_valid_accuracy, fake_rf_valid_recall, fake_rf_valid_precision, \
fake_rf_valid_specificity = print_report(y_valid, fake_y_valid_preds, thresh)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 69)
lr.fit(X_train_tf, y_train)

y_train_preds = lr.predict_proba(X_train_tf)[:,1]
y_valid_preds = lr.predict_proba(X_valid_tf)[:,1]

print('Logistic Regression')
print('Training:')
_ = print_report(y_train,y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
fake_lr = LogisticRegression(random_state = 69)
fake_lr.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_lr.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_lr.predict_proba(X_valid_tf)[:,1]

print('Logistic Regression')
print('Training:')
_ = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid, fake_y_valid_preds, thresh)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 10, random_state = 69)
tree.fit(X_train_tf, y_train)

In [ ]:
y_train_preds = tree.predict_proba(X_train_tf)[:,1]
y_valid_preds = tree.predict_proba(X_valid_tf)[:,1]

print('Decision Tree')
print('Training:')
_ = print_report(y_train,y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
fake_tree = DecisionTreeClassifier(max_depth = 10, random_state = 69)
fake_tree.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_tree.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_tree.predict_proba(X_valid_tf)[:,1]

print('Decision Tree')
print('Training:')
_ = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid, fake_y_valid_preds, thresh)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0, max_depth=3, random_state=69)
gbc.fit(X_train_tf, y_train)

y_train_preds = gbc.predict_proba(X_train_tf)[:,1]
y_valid_preds = gbc.predict_proba(X_valid_tf)[:,1]

print('Gradient Boosting Classifier')
print('Training:')
_ = print_report(y_train,y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
fake_gbc = GradientBoostingClassifier(
    n_estimators=100, learning_rate=1.0, max_depth=3, random_state=69)
fake_gbc.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_gbc.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_gbc.predict_proba(X_valid_tf)[:,1]

print('Gradient Boosting Classifier')
print('Training:')
_ = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid, fake_y_valid_preds, thresh)

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb
xgbc = XGBClassifier()
xgbc.fit(X_train_tf, y_train)

y_train_preds = xgbc.predict_proba(X_train_tf)[:,1]
y_valid_preds = xgbc.predict_proba(X_valid_tf)[:,1]

print('Xtreme Gradient Boosting Classifier')
print('Training:')
_ = print_report(y_train,y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid,y_valid_preds, thresh)

In [ ]:
fake_xgbc = XGBClassifier()
fake_xgbc.fit(fake_X_train_tf, fake_y_train)

fake_y_train_preds = fake_xgbc.predict_proba(X_train_tf)[:,1]
fake_y_valid_preds = fake_xgbc.predict_proba(X_valid_tf)[:,1]

print('Xtreme Gradient Boosting Classifier')
print('Training:')
_ = print_report(y_train, fake_y_train_preds, thresh)
print('Validation:')
_ = print_report(y_valid, fake_y_valid_preds, thresh)